In [1]:
import sys
sys.path.append("utils/")
import wv_util as wv
from utils.dataset_utils import *

In [2]:
coords1, chips1, classes1 = wv.get_labels('/data/zjc4/xView_train.geojson')

100%|██████████| 601937/601937 [00:03<00:00, 163298.70it/s]


In [3]:
idxs = np.arange(0,coords1.shape[0]).reshape(-1,1)
coords1_idx = np.hstack( (coords1, idxs ))


In [4]:
coords1_idx.shape

(601937, 5)

In [5]:
import aug_util as aug
import wv_util as wv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import tqdm
import pickle
import itertools
import cv2 as cv
import glob
import random
import os
import imp

from PIL import Image
from sklearn.model_selection import train_test_split
import dataset_utils as du

imp.reload(wv)
imp.reload(du)

import random
np.random.seed(10)
random.seed(10)
unique_chips = np.unique(chips1)
selected_images = np.random.choice(unique_chips,20)
#selected_images = np.random.choice(unique_chips,10)
idxs = np.isin(chips1,selected_images)

# Input desired classes
grouped_classes = [[77,73],[11,12],[13],[17,18,20,21],
       [19,23,24,25,28,29,60,61,65,26],[41,42,50,40,44,45,47,49]]

labels = ["building and facility" ,"small aircraft", 
          "large aircraft","vehicles","bus","boat"]

xdataset = du.XviewDataset(grouped_classes,labels, coords1_idx[idxs],chips1[idxs],classes1[idxs])
tif_names,distrbs = xdataset.splitTrainTest(debug=True)
dfs,gc = distrbs

string_sets = ["train","valid"]
datum = list(zip(string_sets,tif_names))



"""
Generate darknet images for 30 cm
"""
# Deleting all old images
os.system("rm /data/zjc4/chipped-30/data/images/*")
os.system("rm /data/zjc4/chipped-30/data/labels/*")

dnf = du.DarkNetFormatter(output_dir_ = "/data/zjc4/chipped-30/data/",
                       input_dir_="/data/zjc4/",
                     coords_ = coords1_idx[idxs],
                       chips_ = chips1[idxs],
                       classes_ = classes1[idxs],
                       grouped_classes_=grouped_classes)
dnf.transformDatum(datum,30,showImg=False,chipImage=True)

20it [00:00, 836.86it/s]


In [6]:
dfs[1].groupby(['grouped cls','count']).sum()

og label
grouped cls           count                                                    
boat                  0.0                                    SailboatYachtBarge
                      13.0                                              Tugboat
                      2.0                                        MotorboatFerry
                      4.0                                        Fishing Vessel
                      5.0                                       Maritime Vessel
building and facility 10.0                                             Facility
                      1583.0                                           Building
bus                   0.0     Truck w/FlatbedTruck w/LiquidHaul TruckCement ...
                      3.0                                            Dump Truck
                      4.0                              Truck w/BoxTruck Tractor
                      45.0                                                  Bus
                      46.0                                                Truck
                      9.0                                           Cargo Truck
large aircraft        6.0                                           Cargo Plane
small aircraft        0.0                     Fixed-wing AircraftSmall Aircraft
vehicles              1.0                                         Utility Truck
                      1021.0                                          Small Car
                      4.0                                          Pickup Truck
                      9.0                                     Passenger Vehicle